In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

In [17]:
# Tools to strip HTML
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [18]:
filePath = os.path.join(PROJ_ROOT, "data", "raw", "yale_bios.csv.gz")

In [19]:
data = pd.read_csv(filePath, compression="gzip")
data.shape

(8618, 2)

In [20]:
data.columns = ["A", "B"]
stream = data["B"].str.contains("synopsis clearfix")
stream.isnull().sum()

83

In [21]:
# Preprocessing to remove NAs
new = []
for i in stream:
    if i == True:
        new.append(i)
    else:
        new.append(False)
stream = new
data["BioAvail"] = new

In [22]:
data.head(10)

,A,B,BioAvail
0,0.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
1,1.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
2,10.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
3,100.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
4,1000.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
5,1001.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
6,1002.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
7,1003.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
8,1004.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True
9,1005.0,"<div class=""bio-wrap clearfix"" data-module=""st...",True


In [23]:
def formatBio(row):
    if row["BioAvail"] == True:
        bs = BeautifulSoup(row["B"], 'html.parser')
        rawName = bs.find(class_="name")
        processedName = strip_tags(str(rawName))
        rawBio = bs.find(class_="synopsis clearfix")
        bioText = str(rawBio)
        processedBio = strip_tags(bioText)
        # Returns a tuple (to be formatted later)
        return (processedBio, processedName)
    else:
        return ("")

In [24]:
formattedBios = data.apply(formatBio, axis=1)

In [25]:
formattedBios[100][0]

'\n2007: Played on JV team.\n\n2006: Played on JV team.\n\nAt Germantown Academy: Played soccer for three years before switching to football under captain and former Yale DL Joe Traynor... Nominated for All-Montgomery County All-Star team for football in 2005\n\nPersonal: Benjamin Peter Siegl is the son of Peter and Sharon Siegl...Chemical Engineering major... Interests include golf, skiing and sailing.. devout Philadelphia sports fanatic.\n'

In [26]:
names = []
bios = []

failed = 0
for index, i in enumerate(new):
    if i:
        names.append(formattedBios[index][1])
        bio = re.sub("\n", "", formattedBios[index][0])
        bios.append(bio)
    else:
        names.append("")
        bios.append("")

In [27]:
# players.columns

NameError: name 'players' is not defined

In [ ]:
# players["Name"][-20:]
# players[""]

In [28]:
# Assemble dataframe
namesBios = {
    "Name": names,
    "Bio": bios
    }

nbDataFrame = pd.DataFrame.from_dict(namesBios)

In [29]:
nbDataFrame.shape
# Reorder
nbDataFrame = nbDataFrame[["Name", "Bio"]]
nbDataFrame.head()

,Name,Bio
0,Ryan Brenner,"2009: Made 22 appearances, including 20 starts..."
1,Joe Castaldi,"2010: Team captain.2009: Made 12 appearances, ..."
2,Adam Straus-Goldfarb,2015-16 (Junior): Three seat of the fourth var...
3,Robert Wulbern,2012-13: Rowed in stroke-seat of the 2V at Eas...
4,Matt Coombs,2006 Backup DB who played in all but one game....


In [30]:
namesBiosPath = os.path.join(PROJ_ROOT, "data", "interim", "yale_names_bios.csv.gz")
nbDataFrame.to_csv(namesBiosPath, compression="gzip")